# Description

Script for testing the OpenAI x MuJoCo Simulation Setup of DRL_AI4RoMoCo project with Universal Robot UR10, HEG part and car chassis and comparing with the corresponding reality setup

# Imports

In [1]:
import os
import gym
import pandas
import numpy
import mujoco_py
from gym.envs.robotics import rotations

import rospy
from realsense2_camera.msg import CustomVector
from sensor_msgs.msg import JointState
from realsense2_camera.msg import EstimatedPose

# Paths

In [2]:
HOME = os.getenv("HOME")
MODEL_PATH = os.path.join(*[HOME,"DRL_AI4RoMoCo", "code", "environment" ,"UR10"])
XML_MODEL_PATH = os.path.join(*[MODEL_PATH, "ur10_assembly_setup_rand_temp_1.xml"])

# Configuration

In [4]:
goal = numpy.array([0.69423743, -0.83110109,  1.17388998, -1.57161506,  0.02185773, -3.14102438]) # goal in x,y,z, r,p,y in simulation
q_init = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0.0])
ctrl_q = q_init
ft_values = numpy.array([0, 0, 0, 0, 0, 0])

#final position
q_final = numpy.array([1.8645248413085938, -1.1505220572101038, -0.017417256032125294, -0.7133796850787562, 1.5534274578094482, 9.549929382046685e-05])[[2,1,0,3,4,5]]

# Functions

In [5]:
'''MuJoCo Simulation'''
def set_state(qpos):
    assert qpos.shape == (model.nq,)
    old_state = sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, qpos, old_state.qvel,
                                     old_state.act, old_state.udd_state)
    sim.set_state(new_state)
    sim.forward()
    
def get_sim_obs():
    x_pos = sim.data.get_body_xpos("gripper_dummy_heg")
    x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
    rpy =  normalize_rad(rotations.mat2euler(x_mat))

    obs = numpy.concatenate([
                x_mat.dot(x_pos-goal[:3]), x_mat.dot(normalize_rad(rpy-goal[3:])), ft_values.copy()
            ])
    pose = numpy.concatenate([x_pos-goal[:3], normalize_rad(rpy-goal[3:])])
    #pose = numpy.concatenate([x_pos-goal[:3], rpy-goal[3:]])
    return obs, pose

'''Utility Function'''
def normalize_rad(angles):
    angles = numpy.array(angles)
    angles = angles % (2*numpy.pi)
    angles = (angles + 2*numpy.pi) % (2*numpy.pi)
    for i in range(len(angles)):
        if (angles[i] > numpy.pi):
            angles[i] -= 2*numpy.pi
    return angles

'''ROS Callback Functions'''
def q_callback(data):
    '''Callback receiving the real robot's joint angles'''
    global q_real
    q_real = numpy.array(data.position)[[2,1,0,3,4,5]]

def observation_callback(data):
    '''Callback for the entire observation array (same as in simulation)'''
    global observations
    observations = numpy.array(data.data)

def pose_callback(data):
    '''Callback for the estimated pose'''
    global pose_r
    #pose_r_cor = numpy.concatenate([numpy.array(data.tx, data.ty, data.tz), normalize_rad(numpy.array(data.rx, data.ry, data.rz)])
    pose_r = numpy.array([data.tx, data.ty, data.tz, data.rx, data.ry, data.rz])

In [6]:
def publish_sim_pose(q_sim):
    set_state(q_sim)
    _, pose = get_sim_obs()
    command = EstimatedPose()
    command.tx = pose[0]
    command.ty = pose[1]
    command.tz = pose[2]
    command.rx = pose[3]
    command.ry = pose[4]
    command.rz = pose[5]
    sim_pose_pub.publish(command)
    
def publish_sim_obs(q_sim):
    set_state(q_sim)
    obs, _ = get_sim_obs()
    command = CustomVector()
    command.data = obs
    sim_obs_pub.publish(command)

# Application | Setup

In [7]:
'''MuJoCo'''
model = mujoco_py.load_model_from_path(XML_MODEL_PATH)
sim = mujoco_py.MjSim(model)
set_state(q_init)

'''ROS'''
rospy.init_node("sim_real_eval", anonymous=True)
rospy.Subscriber("/joint_states", JointState, q_callback)
rospy.Subscriber("/observation", CustomVector, observation_callback)
rospy.Subscriber("/pose_estimation", EstimatedPose, pose_callback)
sim_pose_pub = rospy.Publisher("/sim_pose", EstimatedPose, queue_size=1)
sim_obs_pub = rospy.Publisher("/sim_obs", CustomVector, queue_size=1)

In [ ]:
while True:
    publish_sim_obs(q_real)
    publish_sim_pose(q_real)

In [8]:
set_state(q_final)
obs_final, pose_final = get_sim_obs()

In [12]:
pose_final

array([ 6.58733012e-04, -1.68637423e-02,  6.66705471e-03,  9.25054461e-04,
       -2.18092801e-02,  5.47262970e-05])

#### Set Simulation to the same position as the real robot

In [ ]:
q_sim = q_real
set_state(q_sim)

#### Visual control via rendering

In [38]:
viewer = mujoco_py.MjViewer(sim)
while True:
    viewer.render()

Creating window glfw


NameError: name 'exit' is not defined

# Compare observations

In [62]:
obs_sim, pose_sim = get_sim_obs()
obs_real = observations
pose_real = pose_r
obs_deviation = obs_sim[:6] - obs_real[:6]
pose_deviation = pose_sim - pose_real
obs_sim = obs_sim[:6]
obs_real = obs_real[:6]

### Observation

In [63]:
obs_sim

array([ 0.12189322,  0.00363238, -0.00376154, -0.41248823,  0.00833416,
       -0.02017553])

In [64]:
obs_real

array([ 0.12543096, -0.02023591,  0.01335345,  0.17808691, -0.05743716,
        0.0298365 ])

In [65]:
obs_deviation

array([-0.00353773,  0.02386829, -0.01711499, -0.59057513,  0.06577131,
       -0.05001203])

### Pose

In [66]:
pose_sim

array([-0.12189398, -0.00488317,  0.00182055,  0.41248509, -0.02188422,
       -0.00042417])

In [67]:
pose_real

array([-0.1254213 ,  0.02035098, -0.01321027, -0.1780713 ,  0.05035098,
       -0.04071027])

In [68]:
pose_deviation

array([ 0.00352733, -0.02523414,  0.01503083,  0.59055639, -0.07223519,
        0.0402861 ])